In [89]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [118]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [91]:
#spacex_df.drop('Unnamed: 0', axis=1, inplace=True)
spacex_df['Launch Site'].unique()
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [128]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site_dropdown', options=[{'label': 'All Sites', 'value': 'All sites'},
                                                                {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                                {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                                                {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                                {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'}],
                                            value='All Sites',
                                            placeholder="Select the Launch Site",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                
                                # TASK 3: Add a slider to select payload range
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(id='payload_slider', min=0, max=10000, step=1000,
                                                marks={0: '0 kg',
                                            1000: '1000 kg',
                                            2000: '2000 kg',
                                            3000: '3000 kg',
                                            4000: '4000 kg',
                                            5000: '5000 kg',
                                            6000: '6000 kg',
                                            7000: '7000 kg',
                                            8000: '8000 kg',
                                            9000: '9000 kg',
                                            10000: '10000 kg'}, value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [95]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [129]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
Output('success-pie-chart', 'figure'),
Input('site_dropdown', 'value'))

def update_piechart(site_dropdown):
    if site_dropdown=='All sites':
        spacex_df_all = spacex_df.groupby('Launch Site').mean().reset_index()
        fig = px.pie(spacex_df_all, hole=.2, values='class', names='Launch Site', title='Success rate for all sites')
        
    else:
        spacex_dff = spacex_df.loc[spacex_df['Launch Site']==site_dropdown]
        fig = px.pie(spacex_dff, names='class', title='Success rate for '+site_dropdown)
        
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
Output('success-payload-scatter-chart', 'figure'),
Input('site-dropdown', 'value'),
Input('payload_slider', 'value'))

def update_scattergraph(site_dropdown, payload_slider):
    if site-dropdown == 'All Sites':
        low = payload_slider[0]
        high = payload_slider[-1]
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= int(low)) &
                                (spacex_df['Payload Mass (kg)'] <= int(high))
                               ]      
        fig = px.scatter(
            filtered_df, x="Payload Mass (kg)", y="class",
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    else:
        low = payload_slider[0]
        high = payload_slider[-1]
        filtered_df = spacex_df[(spacex_df['Launch Site'] == site_dropdown) & 
                                (spacex_df['Payload Mass (kg)'] >= int(low)) &
                                (spacex_df['Payload Mass (kg)'] <= int(high))
                               ]
        fig = px.scatter(
            filtered_df, x="Payload Mass (kg)", y="class",
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    return fig

In [130]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [08/May/2023 15:42:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2023 15:42:01] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2023 15:42:01] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2023 15:42:02] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [08/May/2023 15:42:02] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [08/May/2023 15:42:02] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [08/May/2023 15:42:02] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [08/May/2023 15:42:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/May/2023 15:42:08] "POST /_dash-update-component HTTP/1.1" 200 -


In [21]:
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [27]:
df = spacex_df.groupby('Launch Site').mean().reset_index()
df.head()

,Launch Site,Flight Number,class,Payload Mass (kg)
0,CCAFS LC-40,14.576923,0.269231,2590.884615
1,CCAFS SLC-40,50.142857,0.428571,3516.664286
2,KSC LC-39A,37.692308,0.769231,4376.511538
3,VAFB SLC-4E,37.600000,0.400000,5813.800000


In [29]:
spacex_df_all = spacex_df.groupby('Launch Site').mean().reset_index()
fig = px.pie(spacex_df_all, values='class', names='Launch Site', title='Success rate for all sites')
fig.show()

In [35]:
site='KCS LC-39A'
spacex_df_filtered = spacex_df[spacex_df['Launch Site']==site]
spacex_df_filtered.head()
#fig = px.pie(spacex_df_filtered, values='class', names='class', title=f"Success rate for {site}")
#fig.show()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category


In [69]:
df = spacex_df[spacex_df['Flight Number']==1]
df = spacex_df[spacex_df['Launch Site']=='KSC LC-39A']
fig = px.pie(df, values='class', names='class')
fig.show()

In [58]:
spacex_df['Launch Site'].unique()

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [78]:
df_2 = df['class'].value_counts()
df_2.values

array([10,  3])

In [79]:
df['class'].value_counts().values

array([10,  3])

In [117]:
fig = px.scatter(spacex_df, x='Payload Mass (kg)', y='class', color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
fig.show()

In [115]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0
